In [ ]:
from llama_index.core import PromptTemplate
from openai import AzureOpenAI

choices = [
    "Useful for questions related to apples",
    "Useful for questions related to oranges",
]


def get_choice_str(choices):
    choices_str = "\n\n".join(
        [f"{idx+1}. {c}" for idx, c in enumerate(choices)]
    )
    return choices_str


choices_str = get_choice_str(choices)

In [3]:
router_prompt0 = PromptTemplate(
    "Some choices are given below. It is provided in a numbered list (1 to"
    " {num_choices}), where each item in the list corresponds to a"
    " summary.\n---------------------\n{context_list}\n---------------------\nUsing"
    " only the choices above and not prior knowledge, return the top choices"
    " (no more than {max_outputs}, but only select what is needed) that are"
    " most relevant to the question: '{query_str}'\n"
)

In [ ]:
llm = AzureOpenAI(
    azure_endpoint="https://oai-tss-sweden.openai.azure.com/",
    api_version="2024-02-01",
    api_key="f0c2aee69c914b35aa55732deb3532be"
)

In [6]:
def get_formatted_prompt(query_str):
    fmt_prompt = router_prompt0.format(
        num_choices=len(choices),
        max_outputs=2,
        context_list=choices_str,
        query_str=query_str,
    )
    return fmt_prompt

In [7]:
query_str = "Can you tell me more about the amount of Vitamin C in apples"
fmt_prompt = get_formatted_prompt(query_str)
print(fmt_prompt)

Some choices are given below. It is provided in a numbered list (1 to 2), where each item in the list corresponds to a summary.
---------------------
1. Useful for questions related to apples

2. Useful for questions related to oranges
---------------------
Using only the choices above and not prior knowledge, return the top choices (no more than 2, but only select what is needed) that are most relevant to the question: 'Can you tell me more about the amount of Vitamin C in apples'



In [13]:
response = llm.predict(fmt_prompt)

C:\Users\Vishwas\AppData\Local\Temp\ipykernel_19572\3675367203.py:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(fmt_prompt)


In [14]:
response

'1. Useful for questions related to apples'

In [12]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
            azure_deployment="gpt-4o-mini",
            api_version="2024-02-01",  
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            azure_endpoint="https://oai-tss-sweden.openai.azure.com/",
            api_key="f0c2aee69c914b35aa55732deb3532be",
            streaming=True
        )

Structured Response in Json

In [15]:
from dataclasses import fields
from pydantic import BaseModel
import json

In [16]:
class Answer(BaseModel):
    choice: int
    reason: str

In [18]:
print(json.dumps(Answer.model_json_schema(), indent=2))

{
  "properties": {
    "choice": {
      "title": "Choice",
      "type": "integer"
    },
    "reason": {
      "title": "Reason",
      "type": "string"
    }
  },
  "required": [
    "choice",
    "reason"
  ],
  "title": "Answer",
  "type": "object"
}


In [22]:
from llama_index.core.types import BaseOutputParser
FORMAT_STR = """The output should be formatted as a JSON instance that conforms to 
the JSON schema below. 

Here is the output schema:
{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "choice": {
        "type": "integer"
      },
      "reason": {
        "type": "string"
      }
    },
    "required": [
      "choice",
      "reason"
    ],
    "additionalProperties": false
  }
}
"""


If we want to put FORMAT_STR as part of an f-string as part of a prompt template, then we'll need to escape the curly braces so that they don't get treated as template variables.

In [23]:
def _escape_curly_braces(input_string: str) -> str:
    # Replace '{' with '{{' and '}' with '}}' to escape curly braces
    escaped_string = input_string.replace("{", "{{").replace("}", "}}")
    return escaped_string

We now define a simple parsing function to extract out the JSON string from the LLM response (by searching for square brackets)

In [25]:
def _marshal_output_to_json(output: str) -> str:
    output = output.strip()
    left = output.find("[")
    right = output.find("]")
    output = output[left : right + 1]
    return output

In [26]:
from typing import List


class RouterOutputParser(BaseOutputParser):
    def parse(self, output: str) -> List[Answer]:
        """Parse string."""
        json_output = _marshal_output_to_json(output)
        json_dicts = json.loads(json_output)
        answers = [Answer.from_dict(json_dict) for json_dict in json_dicts]
        return answers

    def format(self, prompt_template: str) -> str:
        return prompt_template + "\n\n" + _escape_curly_braces(FORMAT_STR)

In [27]:
output_parser = RouterOutputParser()
from typing import List


def route_query(
    query_str: str, choices: List[str], output_parser: RouterOutputParser
):
    choices_str

    fmt_base_prompt = router_prompt0.format(
        num_choices=len(choices),
        max_outputs=len(choices),
        context_list=choices_str,
        query_str=query_str,
    )
    fmt_json_prompt = output_parser.format(fmt_base_prompt)

    raw_output = llm.complete(fmt_json_prompt)
    parsed = output_parser.parse(str(raw_output))

    return parsed

In [28]:
from pydantic import Field


class Answer(BaseModel):
    "Represents a single choice with a reason."
    choice: int
    reason: str


class Answers(BaseModel):
    """Represents a list of answers."""

    answers: List[Answer]

In [31]:
Answers.model_json_schema()

{'$defs': {'Answer': {'description': 'Represents a single choice with a reason.',
   'properties': {'choice': {'title': 'Choice', 'type': 'integer'},
    'reason': {'title': 'Reason', 'type': 'string'}},
   'required': ['choice', 'reason'],
   'title': 'Answer',
   'type': 'object'}},
 'description': 'Represents a list of answers.',
 'properties': {'answers': {'items': {'$ref': '#/$defs/Answer'},
   'title': 'Answers',
   'type': 'array'}},
 'required': ['answers'],
 'title': 'Answers',
 'type': 'object'}

In [30]:
from llama_index.program.openai import OpenAIPydanticProgram
router_prompt1 = router_prompt0.partial_format(
    num_choices=len(choices),
    max_outputs=len(choices),
)
program = OpenAIPydanticProgram.from_defaults(
    output_cls=Answers,
    prompt=router_prompt1,
    verbose=True,
)
query_str = "What are the health benefits of eating orange peels?"
output = program(context_list=choices_str, query_str=query_str)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [33]:
print(router_prompt1)

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['num_choices', 'context_list', 'max_outputs', 'query_str'] kwargs={'num_choices': 2, 'max_outputs': 2} output_parser=None template_var_mappings=None function_mappings=None template="Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.\n---------------------\n{context_list}\n---------------------\nUsing only the choices above and not prior knowledge, return the top choices (no more than {max_outputs}, but only select what is needed) that are most relevant to the question: '{query_str}'\n"


In [ ]:
class Answer(BaseModel)